In [2]:
####################
## Initial stuff
####################

# Load functions, modules and global variables required in our pipeline
from simulate_structures_functions import *
from importlib import reload  
from htmd.builder.charmm import _recoverProtonations

# Are we running curated structures??
curated = True

# PDB codes of the GPCRs to be simulated. 
# If no codes are provided, all avalible structures in GPCRdb will be used (except the ones already simulated)
noncomplex = {"3C9L","3C9M","6ZDR","6ZDV","6TPK", "6S0Q", "6V9S", "6WJC", "6W25", "6OBA", "6LW5", "6KP6", "6LUQ", "6LI1", "6LI0", "6LI2", "6KPC", "6LRY", "6KNM", "6TOT", "6TOS", "6TO7", "6TP6", "6TQ7", "6TOD", "6TP3", "6TQ4", "6TQ9", "6TP4", "6TQ6", "6TPN", "6TPG", "6TPJ", "6OL9", "6RZ6", "6RZ9", "6RZ7", "6RZ8", "6PT2", "6PT3", "6KUX", "6KUY", "6KUW", "6IQL", "6PS7", "6PS0", "6PS3", "6PS5", "6PS1", "6PS4", "6PRZ", "6PS2", "6PS6", "6KK1", "6KK7", "6KJV", "6PS8", "6JZH", "6RZ4", "6RZ5", "6KQI", "6K1Q", "6GT3", "6MH8", "6ME2", "6ME3", "6ME4", "6ME5", "6ME6", "6ME7", "6ME8", "6ME9", "6J21", "6J20", "6A94", "6A93", "5ZTY", "6HLP", "6HLO", "6HLL", "6GPX", "6GPS", "6IIV", "6IIU", "6E59", "6M9T", "6AK3", "5ZHP", "5ZKC", "5ZK3", "5YC8", "5ZK8", "5ZKB", "6IGK", "6IGL", "6FJ3", "6AKX", "6AKY", "6D27", "6D26", "6DRX", "6DS0", "6DRY", "6DRZ", "6BD4", "5ZKQ", "5ZKP", "6C1R", "6C1Q", "6D32", "6D35", "5KW2", "5ZBH", "5ZBQ", "6FK7", "6FKA",  "6FK6", "6FK9", "6FKC", "6FK8", "6FKB", "6CM4", "6FFI", "6FFH", "5WF5", "5WF6", "5V54", "5OLH", "5OLZ", "5OLO", "5OM1", "5OLG", "5OLV", "5OM4", "5YQZ", "6AQF", "5O9H", "5X33", "5VRA", "5WS3", "5WQC", "5WIV", "5WIU", "5NM4", "5NM2", "5NLX", "5X7D", "5X93", "5XPR", "5XSZ", "5N2S", "5MZP", "5MZJ", "5N2R", "5XRA", "5XR8", "5UIW", "5NX2", "5TZY", "5TZR", "5JTB", "5VBL", "5UVI", "5VEW", "5V56", "5V57", "5VEX", "5NDD", "5NDZ", "5UNH", "5UNF", "5UNG", "5TE3", "5TE5", "5UEN", "5UIG", "5TVN", "5T04", "5T1A", "5U09", "5TGZ", "5K2C", "5K2B", "5K2A", "5K2D", "5GLI", "5GLH", "5D6L", "5DYS", "5L7D", "5L7I", "5IU7", "5IUB", "5IU8", "5IU4", "5IUA", "4Z9G", "5EE7", "5DSG", "5CXV", "4ZJ8", "4ZJC", "5F8U", "5DHG", "5DHH", "4ZUD", "5A8E", "5CGC", "5CGD", "4XEE", "4XES", "4Z35", "4Z34", "4Z36", "4YAY", "4UHR", "4XNW", "4XNV", "4XT3", "4RWS", "4RWD", "4S0V", "4U15", "4U16", "4QIM", "4QIN", "4PHU", "4OO9", "4PXZ", "4PY0", "4BVN", "4NTJ", "4OR2", "4O9R", "4BUO", "4N4W", "4N6H", "4NC3", "4MBS", "4L6R", "4K5Y", "4JKV", "3ZPR", "3ZPQ", "4IAQ", "4IAR", "4IB4", "4GPO", "3VW7", "4GBR", "4GRV", "4EIY", "4AMJ", "4AMI", "4EJ4", "4EA3", "3UZA", "3UZC", "4DJH", "4DKL", "3V2Y", "3UON", "3REY", "3RFM", "3RZE", "2YCZ", "2YCW", "2YDV", "2YDO", "3QAK", "2Y04", "2Y00", "2Y03", "2Y02", "3PDS", "3PBL", "3ODU", "3OE0", "3NY9", "3NY8", "3NYA", "3D4S", "2Z73", "2RH1", "1U19", "1GZM"}
noncomplex_nonGPCRmd = {"3C9L","3C9M","6ZDR","6ZDV",'6RZ8', '6FFI', '6KJV', '6TQ6', '6DRY', '5WIV', '5V57', '6LI2', '6MH8', '5MZJ', '5KW2', '5X33', '5ZHP', '6PS6', '6FJ3', '6D35', '4NTJ', '6A94', '6AK3', '5UVI', '6FFH', '5WIU', '6KQI', '5XR8', '6RZ9', '5VEX', '6LW5', '6FKA', '6PS7', '6M9T', '6KNM', '6ME6', '6GPX', '5XSZ', '5K2A', '5YQZ', '6PS0', '5K2C', '6ME7', '2Z73', '6D32', '6RZ4', '5YC8', '5T1A', '6AKY', '4N4W', '5WS3', '6LRY', '6J21', '5TE5', '4O9R', '6FK6', '1GZM', '6DRX', '6TQ7', '5NDZ', '6IGL', '5VRA', '5ZBQ', '6TPN', '5ZTY', '6DS0', '5NLX', '5X93', '6HLP', '5OLG', '5OM4', '5O9H', '5XPR', '6A93', '5GLI', '6OBA', '6D27', '6HLO', '6RZ5', '5TE3', '6IIV', '6KP6', '6TQ4', '6TPJ', '5V56', '5UNG', '6FK9', '6PS8', '2YCZ', '6JZH', '5XRA', '5ZKB', '4JKV', '5MZP', '5K2D', '5OLV', '5X7D', '5T04', '5OLO', '5DYS', '6FK7', '6J20', '6K1Q', '5EE7', '6D26', '6PS1', '6DRZ', '4GBR', '6FKC', '6TP6', '6LUQ', '5ZKQ', '5UNH', '6PRZ', '5UNF', '5NX2', '6LI0', '5NM2', '5D6L', '6AQF', '6KK1', '6C1Q', '4Z9G', '6LI1', '5WF5', '6FKB',  '5OLH', '4XT3', '5ZK3', '5VEW', '4XES', '6HLL', '6TOT', '5N2R', '5WF6', '5K2B', '6TPG', '6PS4', '5F8U', '6ME4', '6GT3', '6RZ6', '6ME8', '6ME2', '5VBL', '5N2S', '5ZBH', '6PS3', '6IIU', '4NC3', '5JTB', '5ZKP', '6TQ9', '6GPS', '6TP3', '6RZ7', '6ME3', '5ZK8', '6C1R', '5V54', '5UIG', '5TVN', '5OM1', '6AKX', '5ZKC', '5NM4', '5TZY', '6TOD', '4GPO', '6TO7', '6CM4', '6TOS', '6PS5', '6ME9', '5NDD', '6KPC', '6KK7', '4BUO', '6OL9', '6PS2', '6FK8', '6ME5', '6TP4', '4QIM', '6IGK', '5WQC', '5UIW', '5TZR', '5OLZ', '4EJ4', '6E59'}
noncomplex_nonGPCRmd_agonist = {'5T04', '6ME9', '4BUO', '6ME6', '5VBL', '5TZR', '6D35', '6LW5', '5YQZ', '6ME3', '5WF6', '6D32', '5TE5', '4XES', '6AK3', '5DYS', '5KW2', '5WF5', '6FK6', '6ME7', '4NC3', '6ME2', '6FKA', '6DRY', '6FK7', '5TVN', '6KPC', '6KQI', '6LRY', '6IGL', '5NX2', '6M9T', '6FKC', '6FKB', '6FJ3', '6IGK', '6KNM', '6FK9', '6ME8', '6ME5', '6ME4', '5XRA', '5N2S', '5TZY', '6FK8', '6LI0', '5XR8'}
pdb_set = noncomplex_nonGPCRmd

# Our main path
basepath = basepath = os.getcwd()+'/'
# Other Paths
strucpath = basepath + 'data_structures/'
resultspath = basepath + 'simulation_output/'
membranepdb = basepath + 'membrane/popc36_box_renumbered.pdb'
topparpath = basepath + 'toppar/TOP_PARAMS_ACE3/'#toppar= topology+parameters
ligandsdict_path = basepath + 'ligands.json'
modres_path = basepath + 'modified_residues.json'
slurmpath = basepath+'fake_slurm/'
# Path to slurm queing system binaries
# In our case, Ismael designed a bunch of small bash scripts (fake_slurm) which do ssh to Hydra and execute slurm there
path= os.environ['PATH']

# Modify path to include fake slurm
%env PATH=$path:$slurmpath

# Load topology, parameter and stream files with our current basepath
topos = [os.path.join(topparpath,file) for file in toposfilenames] 
params = [os.path.join(topparpath,file) for file in paramsfilenames]
streams = [os.path.join(topparpath,file) for file in streamsfilenames]


/soft/system/easybuild/software/Miniconda3/4.7.10/lib/python3.6/site-packages/htmd/versionwarnings.py:30: UserWarning: As of HTMD 1.16 the default number of threads HTMD spawns for calculations is set to 1. You can enable parallelism at your own risk using `config(njobs=-2)` in the beginning of your scripts. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.16');`
  UserWarning,
/soft/system/easybuild/software/Miniconda3/4.7.10/lib/python3.6/site-packages/htmd/versionwarnings.py:38: UserWarning: As of HTMD 1.21 support for ACEMD v2 has stopped. Please use ACEMD3 instead as well as the corresponding equilibration and production protocols. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.21');`
  UserWarning,



Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/



2022-02-14 16:20:23,389 - binstar - INFO - Using Anaconda API: https://api.anaconda.org


New stable HTMD version (2.0.3 python[3.8,<3.9.0a0,3.9,<3.10.0a0,3.7,<3.8.0a0]) is available. You are currently on (1.22.4).There are several methods to update:    - Create a new conda env. using `conda create -n htmd2.0.3 htmd=2.0.3 -c acellera -c psi4 -c conda-forge`    - Create a brand new conda installation and run `conda install htmd -c acellera -c psi4 -c conda-forge`    - Run: `conda update htmd -c acellera -c psi4 -c conda-forge` (NOT RECOMMENDED)



2022-02-14 16:20:25,535 - moleculekit.readers - WARNING - Element guessing failed for atom with name  DUM as the guessed element "D" was not found in the periodic table. Check for incorrect column alignment in the PDB file or report to moleculekit issue tracker.
2022-02-14 16:20:26,202 - moleculekit.readers - WARNING - Element guessing failed for atom with name  DUM as the guessed element "D" was not found in the periodic table. Check for incorrect column alignment in the PDB file or report to moleculekit issue tracker.


env: PATH=/soft/EB_repo/bio/structure/programs/goolf/1.7.20/Chimera/1.11.2/bin:/soft/system/easybuild/software/Miniconda3/4.7.10:/soft/system/easybuild/software/Miniconda3/4.7.10/bin:/soft/EB_repo/bio/structure/programs/foss/2016b/VMD/1.9.4a9/bin:/soft/system/lua/bin:/soft/system/luarocks/bin:/home/daranda/miniconda3/bin:/home/daranda/miniconda3/condabin:/usr/lib64/qt-3.3/bin:/soft/system/lua/bin:/soft/system/luarocks/bin:/usr/local/bin:/usr/local/sbin:/usr/bin:/usr/sbin:/bin:/sbin:/opt/puppetlabs/bin:/home/daranda/bin:/home/daranda/software/getcontacts-master:/usr/local/dowser/bin:/usr/local/dowser/bin/linux:/home/daranda/.local/bin:/home/daranda/bin:/home/daranda/bin:/home/daranda/software/getcontacts-master:/usr/local/dowser/bin:/usr/local/dowser/bin/linux:/gpcr/users/daranda/doctorat/GPCR_simulations/fake_slurm/


In [3]:
################################################
# Part 1: Download data and prepare dictionaries
################################################

# Download and store refined structures from GPCRdb
download_GPCRdb_structures(pdb_set, strucpath)

#Create or moidfy the ligands dictionary
(ligandsdict, ligandsset, modresdict) = ligand_dictionary(pdb_set, ligandsdict_path, modres_path, basepath, blacklist)

# Get topology-parameter files for ligands
get_lig_toppar(ligandsdict, basepath, username, password)

# Get topology-parameter files for modified residues
get_modres_toppar(modresdict, basepath, username, password)

Structure for 5WS3 already present. Skipping...
Structure for 6TP3 already present. Skipping...
Structure for 5T1A already present. Skipping...
Structure for 6J21 already present. Skipping...
Structure for 4O9R already present. Skipping...
Structure for 6RZ7 already present. Skipping...
Structure for 6KP6 already present. Skipping...
Structure for 6D35 already present. Skipping...
Structure for 6RZ5 already present. Skipping...
Structure for 6PRZ already present. Skipping...
Structure for 5K2C already present. Skipping...
Structure for 5UIG already present. Skipping...
Structure for 6TOD already present. Skipping...
Structure for 6LW5 already present. Skipping...
Structure for 4XES already present. Skipping...
Structure for 5X33 already present. Skipping...
Structure for 6D32 already present. Skipping...
Structure for 6FK6 already present. Skipping...
Structure for 6TP6 already present. Skipping...
Structure for 6AKY already present. Skipping...
Structure for 5DYS already present. Skip

something failed in downloading refined structure of 5ZKP: 'NoneType' object has no attribute 'get'
Structure for 6FK8 already present. Skipping...
Structure for 6A94 already present. Skipping...
Structure for 5ZTY already present. Skipping...
Structure for 6ME7 already present. Skipping...
Structure for 5V57 already present. Skipping...
Structure for 4EJ4 already present. Skipping...
Structure for 5WF5 already present. Skipping...
Structure for 5X7D already present. Skipping...
Structure for 6PS3 already present. Skipping...
Structure for 6ZDR already present. Skipping...
Structure for 6IIV already present. Skipping...
Structure for 6JZH already present. Skipping...
Structure for 5XR8 already present. Skipping...
Structure for 5UNG already present. Skipping...
Structure for 5EE7 already present. Skipping...
Structure for 6HLP already present. Skipping...
Structure for 6LI1 already present. Skipping...
Structure for 5T04 already present. Skipping...
Structure for 2Z73 already present. 

Error: File /gpcr/users/daranda/doctorat/GPCR_simulations//data_structures/6KQI/6KQI_refined.pdb was not found.
Error: File /gpcr/users/daranda/doctorat/GPCR_simulations//data_structures/6ZDV/6ZDV_refined.pdb was not found.
Error: File /gpcr/users/daranda/doctorat/GPCR_simulations//data_structures/6TPN/6TPN_refined.pdb was not found.
Error: File /gpcr/users/daranda/doctorat/GPCR_simulations//data_structures/5WIV/5WIV_refined.pdb was not found.
Error: File /gpcr/users/daranda/doctorat/GPCR_simulations//data_structures/4NTJ/4NTJ_refined.pdb was not found.
Error: File /gpcr/users/daranda/doctorat/GPCR_simulations//data_structures/6HLO/6HLO_refined.pdb was not found.
Error: File /gpcr/users/daranda/doctorat/GPCR_simulations//data_structures/6C1R/6C1R_refined.pdb was not found.
Error: File /gpcr/users/daranda/doctorat/GPCR_simulations//data_structures/5V54/5V54_refined.pdb was not found.
Error: File /gpcr/users/daranda/doctorat/GPCR_simulations//data_structures/6LI2/6LI2_refined.pdb was not

file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/HG/5YC8_HG.mol2 does not exist. Skipping....
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/HG/5YC8_HG.mol2 does not exist. Skipping....
Getting toppar file for ligand ZMA 
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/ZMA/5OLG_ZMA.mol2 does not exist. Skipping....
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/ZMA/5OLG_ZMA.mol2 does not exist. Skipping....
Getting toppar file for ligand NA 
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/NA/5OLG_NA.mol2 does not exist. Skipping....
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/NA/5OLG_NA.mol2 does not exist. Skipping....
Getting toppar file for ligand NA 
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/NA/5MZJ_NA.mol2 does not exist. Skipping....
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/NA/5MZJ_NA.mol2 does not exist. Skipping....
Ge

2022-02-14 16:20:27,405 - moleculekit.readers - INFO - Attempting PDB query for 6TPJ


file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/9JU/5ZTY_9JU.mol2 does not exist. Skipping....
Getting toppar file for ligand ZN 
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/ZN/6GPS_ZN.mol2 does not exist. Skipping....
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/ZN/6GPS_ZN.mol2 does not exist. Skipping....
Getting toppar file for ligand F7N 
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/F7N/6GPS_F7N.mol2 does not exist. Skipping....
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/F7N/6GPS_F7N.mol2 does not exist. Skipping....
Getting toppar file for ligand ZMA 
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/ZMA/5UVI_ZMA.mol2 does not exist. Skipping....
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/ZMA/5UVI_ZMA.mol2 does not exist. Skipping....
Getting toppar file for ligand ZMA 
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/Ligands/ZMA/

2022-02-14 16:20:30,737 - moleculekit.molecule - INFO - Removed 8607 atoms. 37 atoms remaining in the molecule.
2022-02-14 16:20:30,795 - moleculekit.molecule - INFO - Removed 0 atoms. 37 atoms remaining in the molecule.


file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/mod_residues/YCM/6TPJ_YCM.mol2 does not exist. Skipping....
file /gpcr/users/daranda/doctorat/GPCR_simulations/toppar/mod_residues/YCM/6TPJ_YCM.mol2 does not exist. Skipping....


FileNotFoundError: [Errno 2] No such file or directory: '/gpcr/users/daranda/doctorat/GPCR_simulations/toppar/mod_residues/YCM/6TPJ_legacy_toppar.str' -> '/gpcr/users/daranda/doctorat/GPCR_simulations/toppar/mod_residues/YCM/pre_6TPJ_legacy_toppar.str'

In [4]:
###########################
## Part 2: Build the models 
###########################
# Iterate by GPCRdb structures to simulate
pdbs_number = len(pdb_set)
i = 0
for pdbcode in {"5ZTY"}:
# for pdbcode in bad_set:
    try:
        for apo in [False, True]:
            
            #Starting simulation
            start_time = time.time()        
            i += 1
            sysname = pdbcode+'_apo' if apo else pdbcode
            mystrucpath = strucpath+pdbcode+'/'

            # Skip if there is already a model build for this
            if os.path.exists(resultspath+'build/'+sysname+'/structure.pdb'):
                print('Build model for '+sysname+' already exists. Skipping...')
                continue

            # Check if simulation is aminergic
            aminergic = gpcrdb_dict[pdbcode]['family'].startswith('001_001')
            adenosine = gpcrdb_dict[pdbcode]['family'].startswith('001_006_001')

            ## Load curated structures
            if curated:
                apo_name = "apo" if apo else "complex"
                curated_filename = str("%sreceptor2curate_output/%s/%s_%s_curated.pdb"%(basepath, pdbcode, pdbcode, apo_name))
                gpcrdb_mol = Molecule(curated_filename)
                # Remove ligand hidrogens in case curators put any
                gpcrdb_mol.remove("not (protein or water or lipid) and element H")
            ## Load non-curated structure
            else:
                (sod2x50, watered_filename) = internal_waters(mystrucpath, pdbcode, gpcrdb_dict, apo)
                gpcrdb_mol = Molecule(watered_filename)
            
            # Add peptide ligand to structure if necessary
            if any([ True for lig in gpcrdb_dict[pdbcode]['ligands'] if lig['type'] in {'protein', 'peptide'} ]) and not (apo):
                add_peplig(watered_filename, pdbcode)
            
            # If the pipeline is running in 'apoform mode', remove any non-protein, non-ion, non-water thing on the system      
            # Delete also sod2x50 (we don't want them here)
            if apo:
                gpcrdb_mol.remove('not (protein or water or ion) or element Na')
                
            # Remove unnecessary ligand molecules: mostly crystalization detergents, quelants, buffers,
            # or post-traductional glicosilations
            gpcrdb_mol.remove('resname '+' '.join(blacklist))
            
            # Remove 2x50Sodium from non-A-class GPCRs
            if not gpcrdb_dict[pdbcode]['family'].startswith('001'):
                gpcrdb_mol.remove('element NA')

            # Get aligned OPM structure
            thickness,opm_mol = get_opm(pdbcode)

            # Ismael's function to add labels (segid) for 'ligand' and 'protein' parts of the system
            gpcrdb_mol_fixed,prot_segids = fix_and_prepare_input(gpcrdb_mol,pdbcode,modresdict,new_pdb_chain)
            
            # If there's any, parameterize and rename covalent-bound ligands
            if not apo:
                (gpcrdb_mol_fixed, covligs) = covalent_ligands(gpcrdb_mol_fixed, pdbcode, ligandsdict)
            else:
                covligs = []
            
            # write file to remember to which chain in the original structure belongs each segment
            segchain_json(gpcrdb_mol_fixed, pdbcode, basepath, prot_segids)
            
            # Align structrues using sequences, and take first one
            alignment_results = sequenceStructureAlignment(gpcrdb_mol_fixed, opm_mol, maxalignments = 1)
            mol_aligned = alignment_results[0] 
            
            #Center to receptor XY
            center = np.mean(mol_aligned.get('coords',sel='chain P'),axis=0)
            mol_aligned.moveBy([-center[0],-center[1],0])
            
            # Prepare protein: asign titration states, flipping side chains of HIS, ASN and GLN; rotate some sidechains, optimize waters, etc.
            # Most of this is done with a HTMD function called proteinPrepare()
            # Skip step if we are working with curators structures
            prepared_mol = mol_aligned if curated else prepare_system(mol_aligned, pdbcode, thickness, sod2x50, aminergic, adenosine)
            
            #Add membrane
            print('Adding membrane...')
            membranemol = Molecule(membranepdb)
            mol_membraned, membrane_resnames, membrane_segids, xreps, yreps = add_membrane(prepared_mol, membranemol,prot_segids,membrane_distance)
            
            # Needed later for equilibration
            with open(mystrucpath+"const_sel.txt",'w') as out: 
                const_sel = 'segid '+' '.join(prot_segids)+' and name C CA N O or not (segid ' + \
                  ' '.join(prot_segids)+' or lipid or water or ions ) and noh or segid ION WAT and noh'
                out.write(const_sel)

            #Solvate
            print('Solvating...')
            mol_solvated = solvate_pdbmol(mol_membraned,membrane_segids,water_thickness,water_margin,buffer=buffer,coldist=coldist,prefix='WT')

            # Check if system has lone-pair hallogen atoms. If it does, use legacy CGenFF parameters
            has_halo = bool(len(gpcrdb_mol_fixed.get('name', 'element Br Cl I and not ion')))
            if has_halo:
                cgenff_par = [topparpath+'David_top_params/parameters/legacy_par_all36_cgenff.prm']
                cgenff_top = [topparpath+'David_top_params/topologies/legacy_top_all36_cgenff.rtf']
            else: 
                cgenff_par = [topparpath+'General_top_params/parameters/par_all36_cgenff.prm']
                cgenff_top = [topparpath+'General_top_params/topologies/top_all36_cgenff.rtf']
            
            #Obtain extra parameters for ligands and modified residues 
            ligstreams=extra_parameters(pdbcode, ligandsdict, modresdict, blacklist, covligs, basepath, has_halo)
            
            # Assignign terminology for cap atoms of protein chain, depending if it is the receptor protein or not
            caps = get_caps(prot_segids, mol_solvated)
            #{'P0': ['first ACE', 'last CT3'], 'P1': ['first ACE', 'last CT3']}

            #Pre-build model
            print('Pre-build...')
            prebuildmol = charmm.build(mol_solvated, 
                                       topo=topos+cgenff_top, 
                                       param=params+cgenff_par,
                                       stream=streams+ligstreams,
                                       caps=caps,
                                       outdir=resultspath+'/pre-build/'+sysname,
                                       ionize=False)

            # Save prebuild model topologies in files, and  store prebuild model in molecule object
            prebuild_psffile = prebuildmol.topoloc
            prebuild_pdbfile = os.path.splitext(prebuildmol.topoloc)[0]+'.pdb'
            prebuildmol = Molecule(prebuild_pdbfile)
            _recoverProtonations(prebuildmol)

            # Checking of aromatic insertions (takes quite a lot fo time)
            print('Checking aromatic insertions...')
            mol_removed,removed_indexes = remove_aromatic_insertions(prebuildmol,prot_segids, outpdb=resultspath+'/pre-build/'+sysname+'/aromatic_check.pdb')

            # Checking of water/lipid ratio
            lipid_num = len(set(prebuildmol.get('resid',sel='segid '+membrane_lipid_segid)))
            solv_num = len(mol_removed.get('index',sel='resname TIP3 and name OH2'))
            if float(solv_num) / lipid_num < 35:
                raise ValueError('Water/lipid ratio lower than 35.')

            #Renumber residues
            print('Renumbering...')
            mol_renumbered = renumber_resid_vmd(mol_removed,'segid '+' '.join(membrane_segids),by=2)

            # Ionizing system
            print('Ionizing...')
            molbuilt = charmm.build(mol_removed,
                                    topo=topos+cgenff_top, 
                                    param=params+cgenff_par,
                                    stream=streams+ligstreams,                        
                                    outdir=resultspath+'/ionize/'+sysname,
                                    saltconc=0.15,
                                    caps=caps)
            build_psffile = molbuilt.topoloc
            build_pdbfile = os.path.splitext(molbuilt.topoloc)[0]+'.pdb'
            molbuilt = Molecule(build_pdbfile)
            _recoverProtonations(molbuilt)

            # Remove clashing rebuild waters
            molbuilt.remove("same residue as (water and within 1 of protein)")
            
            #Building system
            print('Building...')
            molbuilt = renumber_resid_vmd(molbuilt,'segid "WT.*" or segid I',by=2)
            molbuilt = charmm.build(molbuilt, 
                                    topo=topos+cgenff_top, 
                                    param=params+cgenff_par,
                                    stream=streams+ligstreams,                        
                                    outdir=resultspath+'/build/'+sysname,
                                    caps=caps,ionize=False)

            print('End of %s after %s seconds\n' % (sysname, time.time() - start_time))

    except Exception as e:
        print("model "+pdbcode+" could not be build because ",e)

2021-12-30 09:38:20,763 - moleculekit.molecule - INFO - Removed 0 atoms. 4888 atoms remaining in the molecule.
2021-12-30 09:38:20,798 - moleculekit.molecule - INFO - Removed 0 atoms. 4888 atoms remaining in the molecule.
2021-12-30 09:38:23,069 - moleculekit.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2021-12-30 09:38:23,070 - moleculekit.molecule - INFO - Removed 15 atoms. 3684 atoms remaining in the molecule.
2021-12-30 09:38:23,323 - moleculekit.molecule - INFO - Removed 0 atoms. 4888 atoms remaining in the molecule.
2021-12-30 09:38:23,354 - moleculekit.molecule - INFO - Removed 2 atoms. 4886 atoms remaining in the molecule.
2021-12-30 09:38:23,450 - moleculekit.molecule - INFO - Removed 0 atoms. 4886 atoms remaining in the molecule.
2021-12-30 09:38:26,818 - moleculekit.molecule - INFO - Removed 0 atoms. 4886 atoms remaining in the molecule.
/home/david/miniconda

Adding membrane...


2021-12-30 09:38:33,738 - htmd.builder.builder - INFO - Replicating Membrane 3x3
Replicating Membrane: 100%|██████████| 9/9 [00:02<00:00,  4.08it/s]
2021-12-30 09:39:57,781 - moleculekit.molecule - INFO - Removed 2066 atoms. 42862 atoms remaining in the molecule.
2021-12-30 09:39:58,915 - moleculekit.molecule - INFO - Removed 139 residues from appended Molecule due to collisions.


Solvating...


2021-12-30 09:39:59,620 - htmd.builder.solvate - INFO - Using water pdb file at: /home/david/miniconda3/lib/python3.6/site-packages/htmd/share/solvate/wat.pdb


...

Building...


2021-12-30 09:44:13,843 - htmd.builder.charmm - INFO - Writing out segments.
2021-12-30 09:44:20,582 - htmd.builder.builder - INFO - One disulfide bond was added


Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'P', resid: 174, insertion: '', segid: 'P'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'P', resid: 179, insertion: '', segid: 'P'>



2021-12-30 09:44:21,674 - htmd.builder.charmm - INFO - Starting the build.
2021-12-30 09:44:31,029 - htmd.builder.charmm - WARNING - Failed to set coordinates for 1 atoms.
2021-12-30 09:44:31,030 - htmd.builder.charmm - WARNING - Poorly guessed coordinates for 1 atoms.
2021-12-30 09:44:31,031 - htmd.builder.charmm - WARNING - 2 undefined warnings were produced during building.
2021-12-30 09:44:31,032 - htmd.builder.charmm - WARNING - Please check /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/build/5ZTY/log.txt for further information.
2021-12-30 09:44:31,033 - htmd.builder.charmm - INFO - Finished building.


End of 5ZTY after 394.39238953590393 seconds

Build model for 5ZTY_apo already exists. Skipping...


In [4]:
#########################
## Part 3: Equillibration
#########################

for pdbcode in pdb_set:
# for pdbcode in ['5XR8']:
    for apo in [False, True]:
        try:
            modelname = pdbcode+'_apo' if apo else pdbcode
            equildir = resultspath+'equil/'+modelname+'/'
            if os.path.exists(equildir+'output.xtc') or os.path.exists(equildir+'simrunning'):
                print(" structure %s already has been equilibrated" % pdbcode)
                continue

            if not os.path.exists(equildir):
                os.makedirs(equildir)

            # Taking vmd selection line
            const_sel = []
            with open(strucpath+pdbcode+'/const_sel.txt', 'r') as outfile:
                const_sel += outfile.readlines()

            md = define_equilibration(const_sel)
            md.write(resultspath+'build/'+modelname,equildir)

            #Substitute run.sh generated by HTMD by a different one, adapted to the specified path of ACEMD
            with open(equildir + 'run.sh', 'w') as f:
                f.write('#!/bin/bash\n%s > %slog.txt 2>&1' % (acemd_path, equildir))
        
            #Prepare slurm job  
            sq = SlurmQueue()
            sq.envvars = acemd_license
            sq.jobname = 'eql_'+pdbcode
            sq.datadir = None
            sq.partition = 'gpcr_gpu'
            sq.ngpu = 1
            sq.ncpu = 1
            sq.prerun = job_commands(equildir, '/home/daranda/'+modelname+'/')
            sq.memory = 2000
#             sq.exclude = ['arwen','aragorn','bifur']
            sq.nodelist = ['arwen']
    
            # Submit
            sq.submit(equildir)
            
        except Exception as e:
            print("model "+modelname+" could not be send to equilibrate because of ",e)

model 6LI1 could not be send to equilibrate because of  Could not locate any coordinates file in /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/build/6LI1. Please set the Equilibration.acemd.coordinates property to point to the coordinates file
 structure 6LI1 already has been equilibrated
 structure 6ME6 already has been equilibrated
 structure 6ME6 already has been equilibrated
 structure 5OM1 already has been equilibrated
 structure 5OM1 already has been equilibrated
 structure 6LUQ already has been equilibrated
 structure 6LUQ already has been equilibrated
 structure 3C9M already has been equilibrated
 structure 3C9M already has been equilibrated
 structure 6D35 already has been equilibrated
 structure 6D35 already has been equilibrated
 structure 6HLO already has been equilibrated
 structure 6HLO already has been equilibrated
 structure 6ME2 already has been equilibrated
 structure 6ME2 already has been equilibrated
 structure 5V54 already has been equilibrated
 s

2022-01-18 15:57:09,453 - htmd.protocols.equilibration_v2 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2022-01-18 15:57:09,454 - htmd.protocols.equilibration_v2 - INFO - Using user-provided restraints and ignoring constraints and fb_potential
2022-01-18 15:58:21,365 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/equil/6TP3/


 structure 6TP3 already has been equilibrated
 structure 6D26 already has been equilibrated
 structure 6D26 already has been equilibrated
 structure 5TZR already has been equilibrated
 structure 5TZR already has been equilibrated
 structure 6LW5 already has been equilibrated
 structure 6LW5 already has been equilibrated
 structure 6TP6 already has been equilibrated
 structure 6TP6 already has been equilibrated
 structure 5DYS already has been equilibrated
 structure 5DYS already has been equilibrated
 structure 4BUO already has been equilibrated
 structure 4BUO already has been equilibrated
 structure 5OLG already has been equilibrated
 structure 5OLG already has been equilibrated
 structure 4XT3 already has been equilibrated
 structure 4XT3 already has been equilibrated
 structure 6MH8 already has been equilibrated
 structure 6MH8 already has been equilibrated
 structure 4NC3 already has been equilibrated
 structure 4NC3 already has been equilibrated
model 4GPO could not be send to eq

2022-01-18 15:58:52,430 - htmd.protocols.equilibration_v2 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2022-01-18 15:58:52,431 - htmd.protocols.equilibration_v2 - INFO - Using user-provided restraints and ignoring constraints and fb_potential
2022-01-18 16:00:08,888 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/equil/5XR8/


 structure 5XR8 already has been equilibrated
 structure 6RZ4 already has been equilibrated
 structure 6RZ4 already has been equilibrated
 structure 6ME8 already has been equilibrated
 structure 6ME8 already has been equilibrated
 structure 6PS6 already has been equilibrated
 structure 6PS6 already has been equilibrated
 structure 5UNG already has been equilibrated
 structure 5UNG already has been equilibrated
 structure 5V56 already has been equilibrated
 structure 5V56 already has been equilibrated
 structure 5F8U already has been equilibrated
 structure 5F8U already has been equilibrated
 structure 5UIG already has been equilibrated
 structure 5UIG already has been equilibrated
 structure 5OLZ already has been equilibrated
 structure 5OLZ already has been equilibrated
 structure 6E59 already has been equilibrated
 structure 6E59 already has been equilibrated
 structure 5MZJ already has been equilibrated
 structure 5MZJ already has been equilibrated
 structure 5NDZ already has been e

2022-01-18 16:01:28,280 - htmd.protocols.equilibration_v2 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2022-01-18 16:01:28,280 - htmd.protocols.equilibration_v2 - INFO - Using user-provided restraints and ignoring constraints and fb_potential
2022-01-18 16:02:22,617 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/equil/6KQI_apo/


 structure 6TOS already has been equilibrated
 structure 6TOS already has been equilibrated
 structure 6J21 already has been equilibrated
 structure 6J21 already has been equilibrated
 structure 5OLH already has been equilibrated
 structure 5OLH already has been equilibrated
 structure 6DRZ already has been equilibrated
 structure 6DRZ already has been equilibrated
 structure 5XPR already has been equilibrated
 structure 5XPR already has been equilibrated
 structure 6LI0 already has been equilibrated
 structure 6LI0 already has been equilibrated
 structure 6PS7 already has been equilibrated
 structure 6PS7 already has been equilibrated
 structure 5VEX already has been equilibrated
 structure 5VEX already has been equilibrated


2022-01-18 16:02:56,171 - htmd.protocols.equilibration_v2 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2022-01-18 16:02:56,171 - htmd.protocols.equilibration_v2 - INFO - Using user-provided restraints and ignoring constraints and fb_potential
2022-01-18 16:04:25,677 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/equil/5ZTY/
2022-01-18 16:04:55,847 - htmd.protocols.equilibration_v2 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2022-01-18 16:04:55,848 - htmd.protocols.equilibration_v2 - INFO - Using user-provided restraints and ignoring constraints and fb_potential
2022-01-18 16:06:02,014 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/equil/5ZTY_apo/


 structure 6GT3 already has been equilibrated
 structure 6GT3 already has been equilibrated
 structure 6RZ6 already has been equilibrated
 structure 6RZ6 already has been equilibrated
 structure 6PS3 already has been equilibrated
 structure 6PS3 already has been equilibrated
 structure 6A93 already has been equilibrated
 structure 6A93 already has been equilibrated
 structure 6ZDV already has been equilibrated
 structure 6ZDV already has been equilibrated
 structure 6ME4 already has been equilibrated
 structure 6ME4 already has been equilibrated
 structure 5WIV already has been equilibrated
 structure 5WIV already has been equilibrated
 structure 6TQ7 already has been equilibrated
 structure 6TQ7 already has been equilibrated
 structure 6AK3 already has been equilibrated
 structure 6AK3 already has been equilibrated
 structure 5UNF already has been equilibrated
 structure 5UNF already has been equilibrated
 structure 5N2S already has been equilibrated
 structure 5N2S already has been e

In [3]:
#####################
## Part 4: Production
#####################

# Production protocol
md = define_production(timestep, trajperiod)

# If some model should be skipped, put its name here
modelname_skip = {}

# For each PDB 
for pdbcode in ['4EJ4','6LRY']:
# for pdbcode in pdb_set:
#     for apo in [False]:
    for apo in [True, False]:
        # must match with equildir in equilibration launcher code and contain input and output of equilibration.
        modelname = pdbcode+'_apo' if apo else pdbcode
        equildir = '%s/equil/%s/' % (resultspath, modelname)
        # IF equillibration is not avalible
        if not os.path.exists(equildir+'output.xtc'):
            print("replicate %d of structure %s has no avalible equillibration" %(rep, pdbcode))
            continue
        for rep in range(1,repnum+1):
            
            try: 
                    
                # If simulation for this PDB has already been run
                proddir='%sproduction/%s/rep_%d/' % (resultspath, modelname, rep)
                if os.path.exists(proddir+'/output.xtc') or os.path.exists(proddir+'simrunning'):
                    print("replicate %d of structure %s already has been simulated" %(rep, pdbcode))
                    continue

                print('submitting replicate %d of %s' % (rep, pdbcode))
                # directory copy output of equilibration to production input (initial working directory for run_prod.sh).
                md.write(equildir,proddir)

                sq = SlurmQueue()
                sq.envvars = acemd_license
                sq.jobname = modelname+'_pr'+str(rep)
                sq.datadir = None
                sq.partition = 'gpcr_gpu'
                sq.prerun = job_commands(proddir, '/home/daranda/%s_pr_%d/'%(modelname,rep))
                sq.ngpu = 1
                sq.ncpu = 1
                sq.exclude = ['aragorn','arwen','bifur']

                #Substitute run.sh generated by HTMD by a different one, adapted to the specified path of ACEMD
                with open(proddir + 'run.sh', 'w') as f:
                    f.write('#!/bin/bash\n%s >%slog.txt 2>&1' % (acemd_path,proddir))

                sq.submit(proddir)
                
            except Exception as e:
                print("model "+modelname+" could not be send to equilibrate because of ",e)            


replicate 1 of structure 4EJ4 already has been simulated
replicate 2 of structure 4EJ4 already has been simulated
replicate 3 of structure 4EJ4 already has been simulated
submitting replicate 1 of 4EJ4


2022-02-10 14:11:09,787 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (SER 204 CA P P) (SER 204 C P P) (PRO 205 N P P) (PRO 205 CA P P) " with indexes [2602, 2609, 2611, 2615]
2022-02-10 14:11:10,254 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2022-02-10 14:12:01,055 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/production/4EJ4/rep_1/


submitting replicate 2 of 4EJ4


2022-02-10 14:12:19,914 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (SER 204 CA P P) (SER 204 C P P) (PRO 205 N P P) (PRO 205 CA P P) " with indexes [2602, 2609, 2611, 2615]
2022-02-10 14:12:20,372 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2022-02-10 14:13:09,531 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/production/4EJ4/rep_2/


submitting replicate 3 of 4EJ4


...

2022-02-10 14:14:48,405 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (ILE 212 CA P P) (ILE 212 C P P) (GLY 213 N P P) (GLY 213 CA P P) " with indexes [2020, 2035, 2037, 2039]
2022-02-10 14:14:48,901 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2022-02-10 14:15:40,890 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/production/6LRY/rep_1/


submitting replicate 2 of 6LRY


2022-02-10 14:16:00,232 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (ILE 212 CA P P) (ILE 212 C P P) (GLY 213 N P P) (GLY 213 CA P P) " with indexes [2020, 2035, 2037, 2039]
2022-02-10 14:16:00,697 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2022-02-10 14:16:44,572 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/production/6LRY/rep_2/


submitting replicate 3 of 6LRY


2022-02-10 14:17:03,929 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (ILE 212 CA P P) (ILE 212 C P P) (GLY 213 N P P) (GLY 213 CA P P) " with indexes [2020, 2035, 2037, 2039]
2022-02-10 14:17:04,387 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2022-02-10 14:17:48,813 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/production/6LRY/rep_3/


In [ ]:

#####################
## EXTRA: ligand RMSD
#####################
### Assess quality of systems by calculating the RMSD of the ligand/s molecule/s

# Set paths and files
outpath = resultspath+'rmsd_lig.tsv'
if os.path(outpath):
    print('RMSDs already computed. SKipping...')
else:
    # Iterate over pdbcodes 
    results = []
    for pdbcode in pdbcodes: 

        # Iterate over production replicates
        for mytrajid in ["1","2","3"]:

            try:
                sysname = "%s_rep%s"%(pdbcode,mytrajid)

                # Input files of simulation
                files_path = resultspath+'production/%s/rep_%s/'%(name, mytrajid)
                mypdbpath = files_path+'structure.pdb'
                mypsfpath = files_path+'structure.psf'
                mytrajpath = files_path+'output_wrapped.xtc'

                # Skip if no trajectory
                if not os.path.exists(mytrajpath):
                    print("no trajectory replicate %s for system %s avalible. Skipping..."%(mytrajid, sysname))
                    continue

                print('computing ligand RMSD for trajectory %s of system %s' % (mytrajid, sysname))

                # Load trajectory and topology into MDA universe, and select protein atoms
                u = mda.Universe(mypsfpath, mytrajpath)
                ligsel = u.select_atoms("segid LIG and not resname CLR")

                # Compute rmsd, extract its values and put them in corresponding lists and dicts
                R = rms.RMSD(ligsel)
                R.run()
                rmsd = np.mean([ a[2] for a in R.rmsd ])
                results.append((sysname,rmsd)) 

            except Exception as e:
                print("error: system %s failed becasue %s"%(sysname,e))

    # Once everything is done, sort and write RMSD results
    out = open(outpath,'w')
    out.write("Simulated_system\tRMSD_ligand\n")
    results_sorted = sorted(results, key=lambda tup: tup[0])
    for line in results_sorted:
        out.write("%s\t%f\n"%(line[0],line[1])) 
    out.close()

In [7]:

##########################
## Part 5: Wrap Structures
##########################

# Wrap trajectories obtained during production with an htmd command
gpcr_sel = "protein and chain "+new_pdb_chain
# for pdbcode in pdb_set:
for pdbcode in ['4EJ4','6LRY']:
# for pdbcode in testset:
    for apo in [False]:
#     for apo in [True, False]:
        try:
            modelname = pdbcode+'_apo' if apo else pdbcode
            mymol_pdb = Molecule('%sproduction/%s/rep_1/structure.pdb' % (resultspath, modelname))
            for rep in range(1,repnum+1):
            #for rep in [3]:
                start_time = time.time()        
                print('wrapping replicate %d of %s' % (rep, modelname))
                proddir='%sproduction/%s/rep_%d/' % (resultspath, modelname, rep)
                rep = str(rep)

                # To avoid repeating wrapping in Trajectories already wrapped, check the existance of this file
                outname = proddir+'output_wrapped.xtc'
                if os.path.exists(outname):
                    print('replicate already wrapped. Skipping...')
                    continue
                
                # Wrap system
                mymol = Molecule('%sproduction/%s/rep_1/structure.psf' % (resultspath, modelname))
                mymol.read(proddir+'output.xtc')
                mymol.wrap(gpcr_sel)

                # Align frames and write
                mymol.align('all', refmol=mymol_pdb)
                mymol.write(outname)

            print('End of %s after %s seconds\n' % (modelname, time.time() - start_time))
        except Exception as e:
            print("model "+modelname+" could not be wrapped because of ",e)            




wrapping replicate 1 of 4EJ4


/soft/system/easybuild/software/Miniconda3/4.7.10/lib/python3.6/site-packages/moleculekit/align.py:16: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, A), array(float32, 2d, A))
  covariance = np.dot(P.T, Q)
/soft/system/easybuild/software/Miniconda3/4.7.10/lib/python3.6/site-packages/moleculekit/align.py:54: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, C), array(float32, 2d, A))
  all1 = np.dot(all1, rot.T)


wrapping replicate 2 of 4EJ4
wrapping replicate 3 of 4EJ4
End of 4EJ4 after 193.5178680419922 seconds

wrapping replicate 1 of 6LRY
wrapping replicate 2 of 6LRY
wrapping replicate 3 of 6LRY
End of 6LRY after 198.5682544708252 seconds



In [6]:
sublist = [('1191','5WF5',True),
('818','5WF5',False),
('819','2Z73',True),
('820','2Z73',False),
('821','6RZ9',True),
('822','6RZ9',False),
('823','6ME3',True),
('824','6ME3',False),
('825','5ZKQ',True),
('826','5ZKQ',False),
('827','5JTB',True),
('828','5JTB',False),
('829','5VEW',True),
('830','5VEW',False),
('831','5ZBQ',True),
('833','6GT3',True),
('834','6GT3',False),
('835','6KK1',True),
('836','6KK1',False),
('837','6FKC',True),
('838','6FKC',False),
('839','6GPS',True),
('840','6GPS',False),
('841','5VEX',True),
('842','5VEX',False),
('843','6ME2',True),
('844','6ME2',False),
('845','5OM4',True),
('846','5OM4',False),
('847','4NTJ',True),
('848','4NTJ',False),
('849','6DRX',True),
('850','6DRX',False),
('851','6FKB',True),
('852','6FKB',False),
('853','5T04',True),
('854','5T04',False),
('855','4EJ4',True),
('856','4EJ4',False),
('857','5WS3',True),
('858','5WS3',False),
('859','5F8U',True),
('860','5F8U',False),
('861','6HLO',True),
('862','6HLO',False),
('863','6FKA',True),
('864','6FKA',False),
('865','5T1A',True),
('866','5T1A',False),
('867','6TQ7',True),
('868','6TQ7',False),
('869','6RZ4',True),
('870','6RZ4',False),
('871','5V57',True),
('872','5V57',False),
('873','5YQZ',True),
('874','5YQZ',False),
('875','6TPJ',True),
('876','6TPJ',False),
('877','6KJV',True),
('879','6M9T',True),
('880','6M9T',False),
('881','5TE3',True),
('882','6PS5',True),
('883','6PS5',False),
('884','6AQF',True),
('885','6AQF',False),
('886','6FJ3',True),
('887','3C9M',True),
('888','3C9M',False),
('889','6TP3',True),
('890','6TP4',True),
('891','6TP4',False),
('892','6FFH',True),
('893','6FFH',False),
('894','6PS1',True),
('895','6PS1',False),
('896','5KW2',True),
('897','5KW2',False),
('898','4XES',True),
('899','4XES',False),
('900','5X7D',True),
('901','5X7D',False),
('902','4N4W',True),
('903','4N4W',False),
('904','5UIG',True),
('905','5UIG',False),
('906','5N2S',True),
('908','6FK9',True),
('909','6FK9',False),
('910','6AKX',True),
('912','6J21',True),
('913','6J21',False),
('914','5NM2',True),
('915','5NM2',False),
('916','2YCZ',True),
('917','2YCZ',False),
('918','6K1Q',True),
('919','6K1Q',False),
('920','6RZ8',True),
('921','6RZ8',False),
('922','4JKV',True),
('923','4JKV',False),
('924','5TZR',True),
('925','5TZR',False),
('926','6ME6',True),
('927','6ME6',False),
('928','5ZK8',True),
('930','6GPX',True),
('931','6GPX',False),
('932','6A94',True),
('933','6A94',False),
('934','6ME4',True),
('935','6ME4',False),
('936','6RZ7',True),
('937','6RZ7',False),
('938','6ME5',True),
('939','6ME5',False),
('940','5TE5',True),
('941','5TE5',False),
('942','6DRZ',True),
('943','6DRZ',False),
('944','5OLZ',True),
('945','5OLZ',False),
('946','5EE7',True),
('947','5EE7',False),
('948','6ME9',True),
('949','6ME9',False),
('950','4NC3',True),
('951','4NC3',False),
('952','6TOS',True),
('953','6TOS',False),
('954','5V56',True),
('955','5V56',False),
('956','6RZ6',True),
('957','6RZ6',False),
('958','6ZDV',True),
('959','6ZDV',False),
('960','6PS4',True),
('961','6PS4',False),
('962','5WIV',True),
('963','5WIV',False),
('964','6TQ9',True),
('965','6TQ9',False),
('966','5UVI',True),
('967','5UVI',False),
('968','5GLI',True),
('969','6LUQ',True),
('970','6LUQ',False),
('971','6FK8',True),
('972','6FK8',False),
('973','6FFI',True),
('974','6FFI',False),
('975','5WQC',True),
('976','5WQC',False),
('977','5TVN',True),
('978','5TVN',False),
('979','6A93',True),
('980','6A93',False),
('981','6HLL',True),
('982','6HLL',False),
('983','5XR8',True),
('984','5K2B',True),
('985','5K2B',False),
('986','6E59',True),
('987','6E59',False),
('988','5OM1',True),
('989','5OM1',False),
('990','6TQ6',True),
('991','6TQ6',False),
('992','6RZ5',True),
('993','6RZ5',False),
('994','6PS2',True),
('995','6PS2',False),
('996','6TPG',True),
('997','6TPG',False),
('1000','6TP6',True),
('1001','6TP6',False),
('1002','5XSZ',True),
('1003','5DYS',True),
('1004','5DYS',False),
('1005','6TOT',True),
('1006','6TOT',False),
('1007','6KK7',True),
('1008','6KK7',False),
('1009','4XT3',True),
('1010','4XT3',False),
('1011','4BUO',True),
('1012','4BUO',False),
('1015','6D27',True),
('1016','6D27',False),
('1017','5XPR',True),
('1018','5XPR',False),
('1019','6D35',True),
('1020','6D35',False),
('1021','5UNG',True),
('1022','5UNG',False),
('1023','6D32',True),
('1024','6D32',False),
('1025','1GZM',True),
('1026','1GZM',False),
('1027','5XRA',True),
('1028','5XRA',False),
('1029','5UIW',True),
('1030','5UIW',False),
('1031','3C9L',True),
('1032','3C9L',False),
('1033','6TPN',True),
('1034','6TPN',False),
('1035','5ZBH',True),
('1036','5ZBH',False),
('1037','5N2R',True),
('1039','5V54',True),
('1040','5V54',False),
('1041','6J20',True),
('1042','6J20',False),
('1043','6FK7',True),
('1044','6FK7',False),
('1045','6TOD',True),
('1046','6TOD',False),
('1047','5UNF',True),
('1048','5UNF',False),
('1049','6DRY',True),
('1050','6DRY',False),
('1051','5O9H',True),
('1052','5O9H',False),
('1053','5OLG',True),
('1054','5OLG',False),
('1055','6CM4',True),
('1056','6CM4',False),
('1057','6PS3',True),
('1058','6PS3',False),
('1059','5TZY',True),
('1060','5TZY',False),
('1061','5OLH',True),
('1062','5OLH',False),
('1063','4QIM',True),
('1064','4QIM',False),
('1065','6LI1',True),
('1066','6TO7',True),
('1067','6TO7',False),
('1068','6HLP',True),
('1069','6HLP',False),
('1070','5UNH',True),
('1071','5UNH',False),
('1072','6AKY',True),
('1074','6FK6',True),
('1075','6FK6',False),
('1076','6ME8',True),
('1077','6ME8',False),
('1078','5ZHP',True),
('1080','5K2D',True),
('1081','5K2D',False),
('1082','6PS6',True),
('1083','6PS6',False),
('1084','5NM4',True),
('1085','5NM4',False),
('1086','6KPC',True),
('1087','6KPC',False),
('1088','5X33',True),
('1089','5X33',False),
('1090','6DS0',True),
('1091','6DS0',False),
('1092','4GPO',True),
('1093','5D6L',True),
('1094','5D6L',False),
('1095','5X93',True),
('1096','5X93',False),
('1097','6TQ4',True),
('1098','6TQ4',False),
('1099','5ZKC',True),
('1101','4GBR',True),
('1102','4GBR',False),
('1104','6OL9',True),
('1106','5OLO',True),
('1107','5OLO',False),
('1108','5NDD',True),
('1109','5NDD',False),
('1110','6PRZ',True),
('1111','6PRZ',False),
('1112','6MH8',True),
('1113','6MH8',False),
('1114','6KP6',True),
('1115','6D26',True),
('1116','6D26',False),
('1117','6LI2',True),
('1118','5MZP',True),
('1119','5MZP',False),
('1120','5ZK3',True),
('1121','5ZK3',False),
('1122','5NLX',True),
('1123','5NLX',False),
('1124','6ME7',True),
('1125','6ME7',False),
('1126','5MZJ',True),
('1127','6LRY',True),
('1128','6LRY',False),
('1129','5K2C',True),
('1130','5K2C',False),
('1131','6JZH',True),
('1132','6JZH',False),
('1133','6KQI',False),
('1134','6OBA',True),
('1135','6OBA',False),
('1136','5WIU',True),
('1137','5WIU',False),
('1138','6PS8',True),
('1139','6PS8',False),
('1140','5WF6',True),
('1141','5WF6',False),
('1142','5VRA',True),
('1143','5VRA',False),
('1144','6PS7',True),
('1145','6PS7',False),
('1146','6IGK',True),
('1147','6IGK',False),
('1148','5OLV',False),
('1149','5NDZ',True),
('1150','5NDZ',False),
('1151','5K2A',True),
('1152','5K2A',False),
('1153','4O9R',True),
('1154','4O9R',False),
('1155','5ZKP',True),
('1156','5ZKP',False),
('1157','6AK3',True),
('1158','6AK3',False),
('1159','5ZKB',True),
('1160','5ZKB',False),
('1161','4Z9G',True),
('1162','4Z9G',False),
('1163','6ZDR',True),
('1164','6ZDR',False),
('1174','6KJV',False),
('1175','5ZBQ',False)]

In [4]:
def resp_to_dict(resp):
    # Convert a json reponse into a dictionary
    return eval(resp.content.decode('UTF-8').replace('true', 'True').replace('false','False'))

def get_headers(s, subm_id):
    """
    Obtain headers, csrftoken and sessionid from current session object
    """
    sessionid = str(s.cookies['sessionid'])
    csrftoken = str(s.cookies['csrftoken'])
    headers = {
        'Referer' : mainurl+'/dynadb/step1/'+subm_id,
        'Cookie' : 'csrftoken=%s; sessionid=%s' %(csrftoken,sessionid),
        'X-CSRFToken' : csrftoken,
    }
    return(sessionid, csrftoken, headers)
def check_chains(pdbcode, mymol):
    """
    Check how many chains from the original PDB structure remain in mymol structure
    And to which Segments of our structure they correspond
    """
    # Load blosum score matrix to align proteins
    blosum62 = substitution_matrices.load("BLOSUM62")

    # Obtain sequences for original PDB file chains, and classifying them by chainID
    # Also check which segment corresponds to what chain
    pdbmol = Molecule(pdbcode)
    chainseg = {}
    chainset = set(pdbmol.get('chain', sel='protein'))
    pdbmol_segseqs = pdbmol.sequence()
    pdbmol_chainseqs = {}
    for chain in chainset:
        segid = np.unique(pdbmol.get('segid', sel='chain '+chain))[0]
        pdbmol_chainseqs[chain] = pdbmol_segseqs[segid]

    # Merging all protein chains in our systems into a single megachain
    mymol_megachain = ''
    for seg,chain in mymol.sequence().items():
        mymol_megachain = mymol_megachain + chain

    # Aligning sequences from original PDB to simulated PDB megasequence
    # To know which chains from the original PDB are preserved
    chain_present = {}
    segtochain = {}
    # For each chain in the original PDB file
    for chain,seq in pdbmol_chainseqs.items():
        chain_present[chain] = False
        # For each segment in our molecule
        for seg,myseq in mymol.sequence().items():
            # Align our molecule segments to the pdb chains
            aligs = pairwise2.align.localms(seq, myseq, 5,-1, -1, -0.5)
            # Check if any of the alignments has more than 4 score by position
            is_present = any([ (alig.score/(alig.end-alig.start)) > 3.5 for alig in aligs ]) 
            if is_present:
                chain_present[chain] = is_present
                segtochain[seg] = chain

    return (chain_present, segtochain)

def get_uniprot_alignment(mymol, segpos, uniprotkbac, isoform, entry_segs):
    """
    Find out the exact coordinates of each Uniprot segment in our protein
    """

    # Obtain sequence of uniprotkbac
    uniprot_id = "%s-%s"%(uniprotkbac,isoform) if isoform else uniprotkbac 
    response = requests.get("https://www.uniprot.org/uniprot/?query=accession:%s&sort=score&columns=sequence&format=tab"%uniprotkbac)
    uniseq = response.text.split('\n')[1]
    
    # Match this uniprot to our protein segments, and save matches
    for seg,myseq in mymol.sequence().items():
        
        # Align our molecule segments to the pdb chains
        aligs = pairwise2.align.localms(myseq, uniseq, 5,-1, -1, -0.5)
        # For every alignment
        for alig in aligs:
            start = alig.start
            end = alig.end
            # If the alignment has no significant amount of mismatches, keep it as segment
            if (alig.score/(end-start)) > 3.5:
                
                # Find coordinates of this segment
                alig_myseq = alig[0]
                
                # Find start of uniprot sequence in alginment
                if alig_myseq.startswith('-'):
                    starting = 0
                else:
                    starting = start - 1 
                # Find ending of uniprot sequence in alginment            
                if alig_myseq.endswith('-'):            
                    ending = len(myseq) - 1
                else:
                    ending = end - 1 - alig.seqA.count('-')
                
                if uniprotkbac not in entry_segs:
                    entry_segs[uniprotkbac] = []
                    
                # Do not add repeated alignments                
                beg = segpos[seg][starting]
                end = segpos[seg][ending]
                repeated_seg = any([ ((seg['beg']==beg) and (seg['end']==end)) for seg in entry_segs[uniprot_id]])
                if not len(entry_segs[uniprot_id]) or not repeated_seg:
                    entry_segs[uniprot_id].append({
                                'chainid' : mymol.get('chain', 'segid '+seg)[0],
                                'segid' : seg,
                                'beg' : segpos[seg][starting],
                                'end' : segpos[seg][ending]})

    return(entry_segs) 


def get_pdb_info(pdbcode, mymol, ligandsdict, blacklist, apo):
    """
    Get informarion of the system specified in the pdbcode from the RCSB-PDB webpage.
    Mainly uniprot sequences, chainIDs and uniprot codes for the chains
    """
    
    # Make string for selecting lignads in rcsb's api
    ligs = '"'+'","'.join(ligandsdict[pdbcode].keys())+'"'
    gpcr_names = ['ceptor','rhodopsin','smoothened']
    
    #Check which chains from the original PDB structure are preserved in mymol structure
    (chain_present, segtochain) = check_chains(pdbcode, mymol)
    
    # Get residue names in our system
    allresnames = set(mymol.resname)
    
    # Extract information from pdb webpage using api
    ligdict = dict()
    protdict = dict()
    datadict = dict()
    entry_segs = dict()
    segpos = dict()

    # Obtain dictionary of sorted protein resids in each segment
    for seg in set(mymol.get('segid', 'protein')):
        segpos[seg] = list(dict.fromkeys(mymol.get('resid', 'segid '+seg)))
    
    # Get information from PDB api
    pdbdict = requests.get('https://data.rcsb.org/graphql?query={\
        entry(entry_id: "'+pdbcode+'") {\
            exptl {method}\
            polymer_entities {\
                entity_poly{pdbx_strand_id,rcsb_sample_sequence_length}\
                rcsb_polymer_entity{pdbx_description}\
                rcsb_polymer_entity_align{\
                    aligned_regions {length}\
                    reference_database_accession \
                    reference_database_isoform \
                }\
            }\
        }\
        chem_comps(comp_ids: ['+ligs+']) {\
            rcsb_chem_comp_descriptor {InChIKey}\
            chem_comp{id,name}\
        }\
    }').json()['data']
        
    # Determine experimental method used, and use the corresponding id in GPCRmd database
    method = pdbdict['entry']['exptl'][0]['method'].lower()
    if 'x-ray' in method: 
        method_id = 0
        method_segs = 0
    elif 'nmr' in method:
        method_id = 1
        method_segs = 1
    elif 'electron microscopy' == method:
        method_id = 4
        method_segs = 7
    else:
        method_id = 5 # Other method
        method_segs = 6
            
    # Get ligand information and classify it (in case the system actually has ligands)
    if len(pdbdict['chem_comps'])>0:
        for lig in pdbdict['chem_comps']:
            ligandInchi = lig['rcsb_chem_comp_descriptor']['InChIKey']
            ligandResname = lig['chem_comp']['id']
            ligandName = lig['chem_comp']['name']
    
            # Exclude ligands not present in our simulated system
            if ligandResname in allresnames:
                ligdict[ligandResname] = (ligandName,ligandInchi)

            # Beware of covalent-bound retinols
            if (ligandResname=='RET') and ("LYR" in allresnames):
                ligdict["LYR"] = ("Lysine-Retinol","FSIGCSBKSBDRLV-GGOCYBHBSA-N")
                
            # Beware of other covalent-bound ligands
            if ("COV" in allresnames) and (ligandsdict[pdbcode][ligandResname][1]):
                ligdict["COV"] = (ligandName,ligandInchi)
                
    # Extract protein chains information
    recname = ""
    peplig = ("",100000000000000000)
    for poly in pdbdict['entry']['polymer_entities']:

        # Determine if this polymer (chain) is a GPCR
        uniname = poly['rcsb_polymer_entity']['pdbx_description'].lower()
        seqlen = poly['entity_poly']['rcsb_sample_sequence_length']
        chainIds = poly['entity_poly']['pdbx_strand_id'].split(',')
        isgpcr = any([(name in uniname) for name in gpcr_names]) 
        
        # Extract name of the system from pdb info
        if isgpcr:
            uninames = uniname.split(',')
            for unis in uninames:
                if any([(name in unis) for name in gpcr_names]):
                    recname = unis
        else: 
            # Select shortest non-receptor sequence as potential peptide ligand
            if seqlen < peplig[1]:
                peplig = (uniname.split(',')[0],seqlen)
                    
        # Get uniprots present in the system, and their corresponbding segments
        for alreg in poly['rcsb_polymer_entity_align']:
            uniprotkbac = alreg['reference_database_accession']
            isoform = alreg['reference_database_isoform']
            uniprot_id = "%s-%s"%(uniprotkbac,isoform) if isoform else uniprotkbac 
            entry_segs = get_uniprot_alignment(mymol, segpos, uniprotkbac, isoform, entry_segs)
            
            # If this uniprot is present in our system, assign its data to protdict
            if uniprot_id in entry_segs:
                protdict[uniprotkbac] = {
                    'isoform' : isoform if isoform else '1',
                    'isgpcr' : isgpcr,
                    'segs' : entry_segs[uniprot_id],
                    'method' : method_id,
                    'method_segs' : method_segs
                }
    
    # Filter ligdict of blacklisted molecules and ions
    for lig in ligdict:
        if (lig in blacklist) or (len(lig) == 2):
            ligdict.pop(lig)

    ligname = ""
    # Find the ligand molecule, if not apo
    if not apo:
        for lig in ligdict:
            molname = ligdict[lig][0]
            inchikey = ligdict[lig][1]
            if lig != 'CLR': # Take any non-cholesterol, non blacklisted molecule as ligand
                # some molecule names are excessivelly large. Take inchikey instead
                ligname = molname if len(molname) < 51 else inchikey 
            # If there is not any other molecule in the system, nor a potential peptide ligand, take cholesterol
            elif (len(ligdict.keys()) == 1) and not peplig[0]: 
                ligname = ligdict[lig][0]
        
        # If still we have no ligand name, must be peptide
        if not ligname:
            ligname = peplig[0]

    # Make system name
    sysname = recname+" in complex with "+ligname if not apo else recname+" (apoform)"
    
    return (protdict,ligdict,segtochain,method_id,sysname)

def login(s):
    # Login into GPCRmd 
    # Will you go, lassie, go? And we'll all gooo toghetheeeer
    headers = {
        'Cookie': 'csrftoken=ZYa0n3mNBjwBMlpDh4lnbBJQdi7GdlP4',
        'Referer': mainurl+'/accounts/login/',
    }
    datalogin = {
        'username': 'david',
        'password': 'Ameboid',
        'next' : '/accounts/memberpage/',
        'csrfmiddlewaretoken' : 'ZYa0n3mNBjwBMlpDh4lnbBJQdi7GdlP4'
    }
    print('loging into GPCRmd')
    logo = s.post(mainurl+'/accounts/login/', 
               data=datalogin,
               headers=headers)
    return s

def new_submission(s, mainurl):
    """
    Create a new submission entry in GPCRmd
    """

    response_new = s.get(mainurl + '/dynadb/step0/')
    soup = BeautifulSoup(response_new.text, 'html.parser')
    step1_link = soup.find('form',attrs={'id' : 'mainform'}).get('action')
    subm_id = step1_link.split('/')[-2]
    print('new submission %s created'%subm_id)
    return subm_id

def new_step1(s, subm_id, pdbcode, trajperiod, timestep, prodpath, modelfile, method_id, sysname, apo):
    """
    Fullfill and send the step 1 of the new submissionform
    # Looone liiie the fieeeelds of Athenryyyyy...
    # Where once, we watched, the small seabirds flyyyyy....
    """
    # Get headers and stuff
    (sessionid, csrftoken, headers)=get_headers(s, subm_id)
    
    # All the stuff we need to send 
    data_submit = {
        'name' : sysname,
        'type' : 0 if apo else 1, # 0 for apo, 1 for complex
        'pdbid' : pdbcode,
        'description' : 'Classical unbiased (NVT ensemble) complex flexibility assay.',
        'source_type' : method_id,
        'id_dynamics_methods': '1', #Molecular mechanics,
        'software': 'ACEMD3',
        'sversion': 'GPUGRID', # TODO: Is that correct??,
        'ff': 'CHARMM',
        'ffversion': '36m Feb 2016',
        'id_assay_types': 1, # Orthosteric (un)/binding,
        'id_dynamics_membrane_types': 2, # Homogeneus membrane,
        'id_dynamics_solvent_types': 2, # TIP3P solvent,
        'timestep': timestep,
        'delta': (trajperiod*timestep)/1e6, # time/frames
        'add_info': ' Classical unbiased (NVT ensemble) complex flexibility assay.', 
    }
    files_submit = {
        'dynamics' : open(prodpath+'structure.pdb', 'rb'),
        'model' : open(modelfile, 'rb')
    }

    # Submit step 1
    rep = s.post(mainurl+'/dynadb/step1_submit/'+subm_id+'/',
            headers = headers,
            files = files_submit,
            data = data_submit)
    print('step1 finalized ',rep)

    # Raise exceptions if something failed
    if not rep.ok:
        #s.post(mainurl+'/dynadb/delete_submission/'+subm_id)
        raise Exception('Step 1 submitted returned %d'%rep.status_code)


    
def smalmol_getdata(s, molid, inchikey, subm_id, moltype, resname, sdfpath=""):
    """
    Get data for small molecules 
    """
    
    # Get headers and stuff
    (sessionid, csrftoken, headers)=get_headers(s, subm_id)

    # Get info of molecule
    rep = s.get(mainurl+'/dynadb/smalmol_info/?inchikey=%s&submission_id=%s'%(inchikey,subm_id),
        headers = headers,
        )
    rep_dict = rep.json()
    
    # Prepare submission dictionary with the data of this molecule
    mol_datasubmit = {
        "smalmol_type"+molid : moltype,
        "smalmol_inchikey"+molid : inchikey,
        "smalmol_name"+molid : rep_dict['name'],
        "smalmol_resname"+molid : resname,
        "smalmol_iupac"+molid : rep_dict['iupac'],
        "smalmol_chemblid"+molid :  rep_dict['chemblid'],
        "smalmol_cid"+molid :  rep_dict['cid'],
        "smalmol_inchi"+molid : rep_dict['inchi'],
        "smalmol_sinchi"+molid :  rep_dict['sinchi'],
        "smalmol_sinchikey"+molid :  rep_dict['sinchikey'],
        "smalmol_smiles"+molid :  rep_dict['smiles'],
        "smalmol_netcharge"+molid :  rep_dict['net_charge'],
        "smalmol_synonims"+molid :  rep_dict['other_names'],
        "smalmol_description"+molid :  '',
        "image_path"+molid :  rep_dict['imagepath']
    }
    
    # Add SDFfile to the submission if required
    mol_files = {} if rep_dict['inGPCRmd'] else {"sdfmol"+molid : open(sdfpath)} 
    
    return(mol_datasubmit,mol_files)

def new_step2(s, subm_id, ligdict, apo):
    """
    Fullfil and send step2 of new submission form
    """    
    print('initiating step 2: small molecule data')

    # Get headers and stuff
    (sessionid, csrftoken, headers)=get_headers(s, subm_id)
    
    # Introduce common small molecules (waters, lipids and ions)
    i = 0
    num_entries = []
    step2_data = {
        'csrfmiddlewaretoken' : csrftoken,
        'submit' : 'submit'
    }
    step2_files = {}
    common_mols = {
        'TIP3' : ('XLYOFNOQVPJJNP-UHFFFAOYSA-N',6),
        'POPC' : ('WTJKGGKOPKCXLL-VYOBOKEXSA-N',7),
        'SOD' : ('FKNQFGJONOIPTF-UHFFFAOYSA-N',8),
        'CLA' : ('VEXZGXHMUGYJMC-UHFFFAOYSA-M',8)
    }
    for resname,(inchikey,moltype) in common_mols.items():
        (mol_datasubmit, mol_files) = smalmol_getdata(s, str(i), inchikey, subm_id, moltype, resname)
        step2_data.update(mol_datasubmit)
        step2_files.update(mol_files)
        num_entries.append(i)
        i+=1

    # Introduce ligands: all will be defined as orthosteric ligands
    lignames = []
    for lig,(name,inchikey) in ligdict.items():
        # Skip this if molecule is apoform
        if apo:
            break
        # Avoid blacklisted molecules or ions
        if (lig in detergent_blacklist) or (lig in glucids_blacklist)  or (len(lig) == 2):
            continue
        # Download ligand, store it into temporary file
        response = requests.get('https://files.rcsb.org/ligands/view/'+lig+'_ideal.sdf')
        with open('tmpfile.sdf','wb') as tmpout:
            tmpout.write(response.content)

        # CLR (cholesterol) is usually a experimental lipid 
        moltype = '3' if lig=='CLR' else '0'
            
        # Retrieve molecule information
        (mol_datasubmit, mol_files) = smalmol_getdata(s, str(i), inchikey, subm_id, moltype, lig, 'tmpfile.sdf')
        step2_data.update(mol_datasubmit)
        step2_files.update(mol_files)
        num_entries.append(i)
        i+=1
        os.remove('tmpfile.sdf')
    
    # Add number of entries
    step2_data['num_entries'] = ','.join(map(str, num_entries))
    
    # Replace empty entries with an empty string
    for key,val in step2_data.items():
        if not val:
            step2_data[key] = ''
    
    # Submit step2
    rep = s.post(mainurl+'/dynadb/step2_submit/'+subm_id+'/',
        headers = headers,
        data = step2_data,
        files = step2_files)
    print('step2 finalized ',rep)

    # Raise exceptions if something failed
    if not rep.ok:
        #s.post(mainurl+'/dynadb/delete_submission/'+subm_id)
        raise Exception('Step 2 submitted returned %d'%rep.status_code)

def new_step3(s, subm_id, pdbcode, protdict):
    """
    Fullfil and sent the step3 (protein chains) of the new submission form
    """
    print('initiating step 3: protein chains')

    # Get headers and stuff
    (sessionid, csrftoken, headers)=get_headers(s, subm_id)
    
    # Start dict with the data to be send into GPCRmd
    # But thougth they paved the footways here with goldust, I still would chose... my Isle of Innesfree
    step3_data = {
        'csrfmiddlewaretoken':csrftoken,
        'submission_id' : subm_id,
        'submit' : 'Submit',
    }

    # Iterate over pdb chains. For each one we'll create an entry
    i=0
    entrynum = []
    for uniprotkbac,entry_data in protdict.items():
        ii = str(i)
        isgpcr = entry_data['isgpcr']
        isoform = entry_data['isoform']
        segs = entry_data['segs']
        method_segs = entry_data['method_segs']

        # Retrieve info using uniprot code
        unidict = s.get(mainurl+'/dynadb/prot_info/'+subm_id+'/',
                          params = {
                              'uniprotkbac' : uniprotkbac,
                              'isoform' : isoform,
                              'submission_id' : subm_id,
                          },
                          headers = headers
                         ).json()

        # We'll obtain the alignment of our protein sequence and its UNIPROT sequence using
        # GPCRmd's tool for do it
        data_alig = {
            'unisequence' : unidict['Sequence'],
            'submission_id' : subm_id
        }

        # Get segments for alignment data
        j = 0
        segnums = []
        for seg in segs:
            ss = str(j)
            data_alig['chain'+ss]= seg['chainid']
            data_alig['segid'+ss]= seg['segid']
            data_alig['from'+ss]= seg['beg']
            data_alig['to'+ss]= seg['end']
            segnums.append(ss)

            # Store segment information as well in the submit dict
            step3_segdata = {
                ss+'seg'+ii: j,
                ss+'pdbid'+ii: pdbcode,
                ss+'sourcetype'+ii: method_segs,
                ss+'chain'+ii: seg['chainid'],
                ss+'segid'+ii: seg['segid'],
                ss+'from_resid'+ii: seg['beg'],
                ss+'to_resid'+ii: seg['end'],
            }
            step3_data.update(step3_segdata)

            j += 1
        data_alig['segnums']=','.join(segnums)
        step3_data['num_segs'+ii]=','.join(segnums)

        # Send alignment request
        alignment = s.post(mainurl+'/dynadb/get_alignment/',
                      data = data_alig,
                      headers = headers
                     ).text

        # Get mutations
        mutations_dict = s.post(mainurl+'/dynadb/protein/get_mutations/',
                      data = {
                          'alignment' : alignment,
                          'sequence' : unidict['Sequence'],
                      },
                      headers = headers
                     ).json()['mutations']

        # Store mutations in submission dictionary
        h = 0
        mutnums = []
        for mut in mutations_dict:
            hh = str(h)
            step3_mutdata = {
                hh+'from'+ii : mut['from'],
                hh+'to'+ii : mut['to'],
                hh+'resid'+ii : mut['resid'],
            }
            step3_data.update(step3_mutdata)
            mutnums.append(hh)
            h+=1
        step3_data['num_muts'+ii]=','.join(mutnums)

        # Get chain name according to uniprot
        names_array = unidict['Protein names'].split(" (")
        names_array = list(map(lambda x: re.sub("\)$|\) \[.*\]$", "", x), names_array)) # Convert the name string into name aray
        name = names_array.pop(0) # We get the first name as the official one
        other_names = ';'.join(names_array)
        
        # Start storing information to send in step3
        step3_entrydata = {
            'isoform'+ii : isoform,
            'prot_uniprot'+ii : uniprotkbac,
            'name'+ii : name,
            'aliases'+ii : other_names,
            'species_code'+ii : unidict['Organism ID'],
            'species_name'+ii : unidict['Organism'],
            'unisequence'+ii : unidict['Sequence'],
        }
        step3_data.update(step3_entrydata)

        entrynum.append(ii)    
        i+=1

    # Store number of entries
    step3_data['num_entries'] = ','.join(entrynum)

    # Submit step3
    rep = s.post(mainurl+'/dynadb/step3_submit/'+subm_id+'/',
        headers = headers,
        data = step3_data)
    
    print('step3 finalized ',rep)

    # Raise exceptions if something failed
    if not rep.ok:
        #s.post(mainurl+'/dynadb/delete_submission/'+subm_id)
        raise Exception('Step 3 submitted returned %d'%rep.status_code)

def other_files_zip(prodpath):
    """
    Compress protocol files in the path submitted 
    """
    outfolder = prodpath+'other_files'
    outfile = outfolder+'.tar.gz'
    if not os.path.exists(outfile):
        os.makedirs(outfolder)
        # For each replicate, create a folder with symbolic links of the to-compress files
        for rep in [1,2,3]:
            repath = '%srep_%d/' % (prodpath,rep)
            outrep = '%s/rep_%d/' % (outfolder,rep)
            os.makedirs(outrep)
            list_files = ['input', 'input.coor', 'input.vel','input.xsc','log.txt']
            for file in list_files:
                shutil.copyfile(repath+file, outrep+file)

        # Compress file
        shutil.make_archive(outfolder, 'gztar', outfolder)
        shutil.rmtree(outfolder)

    return(outfile)

def new_step4(s, subm_id, prodpath, repath):
    """
    Fullfil and sent the step4 (files) of the new submission form
    """
    print('initiating step 4: simulation files')

    # Get headers and stuff
    (sessionid, csrftoken, headers)=get_headers(s, subm_id)

    # Prepare trajectories
    step4_files = []
    for trajid in range(1,4):
        trajpath = '%s/rep_%d/output_wrapped.xtc'%(prodpath,trajid)
        step4_files.append(('trj', ('traj%d.xtc'%trajid, open(trajpath, 'rb'))))
    
    # Prepare the rest of files (parameters files will name-changed to avoid problems with filextensions)
    step4_files.append(
        ('dyn', ('structure.psf', open(repath+'structure.psf','rb')))
    )
    step4_files.append(
        ('prm', ('parameters.prm', open(repath+'parameters','rb')))
    )
    other_files = other_files_zip(prodpath)
    step4_files.append(
        ('oth', ('other_files.tar.gz', open(other_files,'rb')))
    )
    
    # Encode data
    m=MultipartEncoder(fields=step4_files)
    headers['Content-Type'] = m.content_type
    
    # Submit step4
    rep = s.post(mainurl+'/dynadb/step4_submit/'+subm_id+'/',
        headers = headers,
        data = m)
    
    print('step4 finalized ',rep)

    # Raise exceptions if something failed
    if not rep.ok:
        s.post(mainurl+'/dynadb/delete_submission/'+subm_id)
        raise Exception('Step 4 submitted returned %d'%rep.status_code)
    # If everything went ok so far, write a "submitted" file to know this system has already been submitted
    else:
        with open(prodpath+'submitted.txt','w') as out:
            out.write(subm_id)

def check_completeness(prodpath):
    """
    Check if we actually have the three necessary replicates for submission
    """

    incomplete = False
    for trajid in range(1,4):
        trajpath = '%s/rep_%d/output_wrapped.xtc'%(prodpath,trajid)
        if not os.path.exists(trajpath):
            incomplete = True

    return(incomplete)            

In [9]:
#### Actual Simulation submission
# mainurl = 'http://localhost:8000' 
mainurl = 'https://submission.gpcrmd.org'

## Step -2: Login into GPCRmd
with requests.Session() as s:
    login(s)
# For each of the currently-working-with systems defined in Part 1
# for pdbcode in pdb_set:
# for pdbcode in ['6D32']:
bad_list = [('856','4EJ4',False),('1128','6LRY',False)]
for (subm_id,pdbcode, apo) in bad_list:
#     for apo in [False]:
#     for apo in [True, False]:
        
        try:
            # Set the path to the folder containing this simulation's files
            sysname = pdbcode+"_apo" if apo else pdbcode+"_complex" 
            prodpath = resultspath+'production/'+pdbcode+'_apo/' if apo else resultspath+'production/'+pdbcode+'/' 
            repath = prodpath+'rep_1/'# For files no changing across representations
            modelfolder = "%sreceptor2curate_output/%s/"%(basepath, pdbcode) 
            modelpath = "%s%s_apo_curated.pdb"%(modelfolder,pdbcode) if apo else "%s%s_complex_curated.pdb"%(modelfolder,pdbcode)
            
            # Skip if not all trajectories or already submitted or if already submitted
            incomplete = check_completeness(prodpath)
            submitted = os.path.exists(prodpath+'submitted.txt')
            #if incomplete or submitted:
            #    continue            
            
            # Load molecule
            if not os.path.exists(repath+'structure.pdb'):
                continue
            mymol = Molecule(repath+'structure.pdb')
            print('\n###############Submitting '+sysname+' simulation ########################')            

            ## Step -1: Get information of protein chains and ligand molecules from PDB web
            (protdict,ligdict,segtochain,method_id,sysname) = get_pdb_info(pdbcode, mymol, ligandsdict, blacklist, apo)
            
            ## Step 0: Create a new submission
#             subm_id = new_submission(s, mainurl)
            print(subm_id, pdbcode, apo)
#             subm_id = "969"

            ## Step 1: General information
#             new_step1(s, subm_id, pdbcode, trajperiod, timestep, repath, modelpath, method_id, sysname, apo)
            
            ## Step 2: Small molecules 
#             new_step2(s, subm_id, ligdict, apo)   

            ## Step 3: Protein chains
#             new_step3(s, subm_id, pdbcode, protdict)

            ## Step 4: Dynamics information
            new_step4(s, subm_id, prodpath, repath)
    
        except Exception as e:
            print("Simulation "+sysname+" could not be submitted because ",e)            


loging into GPCRmd


2022-02-16 10:53:26,630 - moleculekit.readers - INFO - Attempting PDB query for 4EJ4



###############Submitting 4EJ4_complex simulation ########################
856 4EJ4 False
initiating step 4: simulation files
step4 finalized  <Response [200]>


2022-02-16 10:55:12,159 - moleculekit.readers - INFO - Attempting PDB query for 6LRY



###############Submitting 6LRY_complex simulation ########################
1128 6LRY False
initiating step 4: simulation files
step4 finalized  <Response [200]>


In [5]:
for dyn_id in range(686, 1034): 
    workbench_reponse = requests.get('https://submission.gpcrmd.org/view/'+str(dyn_id))
    report_reponse = requests.get('https://submission.gpcrmd.org/dynadb/dynamics/id/'+str(dyn_id))                       
    if not (workbench_reponse.ok and report_reponse.ok):
        print("Dyn_id: ",dyn_id)
        print("Workbench: ", workbench_reponse)
        print("Report: ", report_reponse)
        print("\n")



Dyn_id:  701
Workbench:  <Response [500]>
Report:  <Response [500]>




KeyboardInterrupt: 